In [ ]:
!wget "https://raw.githubusercontent.com/NVIDIA/TensorRT/main/quickstart/IntroNotebooks/helper.py"

In [1]:
!nvidia-smi

Wed Jun  1 10:04:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.94       Driver Version: 470.94       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   41C    P0   105W / 350W |      0MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:82:00.0 Off |                  N/A |
| 30%   

In [1]:
import tensorflow as tf

print(tf.__version__)

2.6.0


In [8]:
from tensorflow.keras.applications import ResNet50

model_dir = 'data/resnet50_saved_model'
model = ResNet50(include_top=True, weights='imagenet')

In [3]:
model.save(model_dir) 

2022-06-01 09:50:32.022296: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: data/resnet50_saved_model/assets


/root/miniconda3/envs/trt_tf/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [2]:
import numpy as np

BATCH_SIZE = 32

dummy_input_batch = np.zeros((BATCH_SIZE, 224, 224, 3))

In [3]:
PRECISION = "FP32" # Options are "FP32", "FP16", or "INT8"

In [5]:
from trt_help import ModelOptimizer # using the helper from <URL>

model_dir = 'data/resnet50_saved_model'

opt_model = ModelOptimizer(model_dir)
model_fp32 = opt_model.convert(model_dir+'_FP32', precision=PRECISION)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


2022-06-01 10:28:41.148099: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-01 10:28:42.366953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-01 10:28:42.368525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6
2022-06-01 10:28:49.111567: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, co

INFO:tensorflow:Assets written to: data/resnet50_saved_model_FP32/assets


In [6]:
model_fp32.predict(dummy_input_batch)

2022-06-01 11:51:30.868472: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-01 11:51:33.736676: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2022-06-01 11:51:35.161702: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-01 11:51:36.674890: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       ...,
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04]], dtype=float32)

In [9]:
# Warm up - the first batch through a model generally takes longer
model.predict(dummy_input_batch)
model_fp32.predict(dummy_input_batch)

array([[1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       ...,
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04]], dtype=float32)

In [10]:
%%timeit

model.predict_on_batch(dummy_input_batch)

46.5 ms ± 679 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit

model_fp32.predict(dummy_input_batch)

28.8 ms ± 73.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
model_fp16 = opt_model.convert(model_dir+'_FP16', precision="FP16")

model_fp16.predict(dummy_input_batch)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


2022-06-01 11:52:54.188819: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-06-01 11:52:54.188922: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-06-01 11:52:54.195377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-01 11:52:54.196487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6
2022-06-01 11:52:54.243294: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1202 nodes (878), 1857 edges (1533),

INFO:tensorflow:Assets written to: data/resnet50_saved_model_FP16/assets


array([[1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       ...,
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04]], dtype=float32)

In [14]:
%%timeit

model_fp16.predict(dummy_input_batch)

29.1 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
dummy_calibration_batch = np.zeros((8, 224, 224, 3))

opt_model.set_calibration_data(dummy_calibration_batch)

In [16]:
model_int8 = opt_model.convert(model_dir+'_INT8', precision="INT8")

model_int8.predict(dummy_input_batch)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


2022-06-01 11:54:17.504750: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-06-01 11:54:17.504851: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-06-01 11:54:17.511255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-01 11:54:17.512365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6
2022-06-01 11:54:17.547385: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1202 nodes (878), 1857 edges (1533),

INFO:tensorflow:Assets written to: data/resnet50_saved_model_INT8/assets


array([[1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       ...,
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04]], dtype=float32)

In [17]:
%%timeit

model_int8.predict(dummy_input_batch)

36.1 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


: 